In [1]:
import numpy as np 
import MDAnalysis as md
from MDAnalysis.analysis import align
from tqdm.notebook import tqdm
from MDAnalysis.lib import transformations, mdamath
import sys
import scipy.optimize
import itertools
import tqdm.notebook as tqdm
from MDAnalysis.analysis.distances import dist
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
#from termcolor import colored
import os
import subprocess
import re

In [18]:
script_dir       = '/data2/lisbeth_is_awesome/STEROLS/TMs/index_test'

for dirr in os.listdir('/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP'):        

        print ('Entering folder:', dirr)
            
        if re.match('job', dirr):
            continue
        if re.match('run', dirr):
            continue
        if re.match('submit', dirr):
            continue

        
        #Make trajectory whole first
        
        os.chdir('{}/{}'.format('/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP',dirr))
        
        gro = 'r.gro'
        u = md.Universe(gro)
        lipidsagg=u.select_atoms('all and not name NA CL W')
        solventagg=u.select_atoms('name NA CL W')
        lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
        solventagg.write("index.ndx", mode="a", name= 'Solvent')
        u.atoms.write("index.ndx", mode="a", name= 'System')

        
        if os.path.isfile('p_lipids_whole.xtc')==False:
            p = subprocess.Popen('gmx trjconv -f p*xtc -s p.tpr -o p_lipids_whole.xtc -n index.ndx -pbc whole', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
            p.communicate('Lipids\nLipids')
        if os.path.isfile('p_lipids.tpr')==False:
            p = subprocess.Popen('gmx convert-tpr -s p.tpr -o p_lipids.tpr -n index.ndx', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
            p.communicate('Lipids')
        if os.path.isfile('p_lipids.gro')==False:
            p = subprocess.Popen('gmx editconf -f p_lipids.tpr -o p_lipids.gro', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
            
        #os.system('conda activate STEROLS')
        cmd = 'python3 {}/calc_lindemann_ori.py {}.txt p_lipids_whole.xtc p_lipids.gro 1 20000 100'.format(script_dir, dirr)

        file_run = open('run.sh' ,'w')
        file_run.write(cmd)
        file_run.close()

Entering folder: DSPC_90_CAMP_10_box_15_15_rep2


                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'Lipids'


Last frame      20000 time 2000000.000    ->  frame  20000 time 2000000.000      


GROMACS reminds you: "Here's the Way It Might End" (G. Michael)



In [2]:
script_dir       = '/data2/lisbeth_is_awesome/STEROLS/TMs/index_test'
systems = ['DSPC_85_CAMP_15_box_15_15_rep1']
for sys in systems:
    
        os.chdir('/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP/{}'.format(sys))
        
        gro = 'r.gro'
        u = md.Universe(gro)
        lipidsagg=u.select_atoms('all and not name NA CL W')
        solventagg=u.select_atoms('name NA CL W')
        lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
        solventagg.write("index.ndx", mode="a", name= 'Solvent')
        u.atoms.write("index.ndx", mode="a", name= 'System')

        os.system('rm p_lipids*')
        
        p = subprocess.Popen('gmx trjconv -f p*xtc -s p.tpr -o p_lipids_whole.xtc -n index.ndx -pbc whole', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
        p.communicate('Lipids\nLipids')
        p = subprocess.Popen('gmx convert-tpr -s p.tpr -o p_lipids.tpr -n index.ndx', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
        p.communicate('Lipids')
        p = subprocess.Popen('gmx editconf -f p_lipids.tpr -o p_lipids.gro', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
            
        cmd = 'python3 {}/calc_lindemann_ori.py {}.txt p_lipids_whole.xtc p_lipids.gro 1 20000 100'.format(script_dir, f'/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP/{sys}')

        file_run = open('run.sh' ,'w')
        file_run.write(cmd)
        file_run.close()

/home/lravnkilde/.conda/envs/neglview/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: G
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/lravnkilde/.conda/envs/neglview/lib/python3.10/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: R
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
rm: cannot remove 'p_lipids*': No such file or directory
                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'Lipids'
100000000 steps (2e+06 ps) remaining from first run.
Selected 0: 'Lipids'


                   :-) GROMACS - gmx convert-tpr, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
   

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 8316 atoms
Volume: 2854.12 nm^3, corresponds to roughly 1284300 electrons
Velocities found


In [4]:
script_dir       = '/data2/lisbeth_is_awesome/STEROLS/TMs/index_test'
STs = ['CAMP','ERG','STIG','SITO']

for st in STs:
    for rep in range(1,5):
        sys = f'DSPC_95_{st}_5_box_15_15_rep{rep}'
# systems = ['DSPC_85_CAMP_15_box_15_15_rep1']
# for sys in systems:
    
        os.chdir('/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP/{}'.format(sys))
        
        gro = 'r.gro'
        u = md.Universe(gro)
        lipidsagg=u.select_atoms('all and not name NA CL W')
        solventagg=u.select_atoms('name NA CL W')
        lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
        solventagg.write("index.ndx", mode="a", name= 'Solvent')
        u.atoms.write("index.ndx", mode="a", name= 'System')

        os.system('rm p_lipids*')
        
        p = subprocess.Popen('gmx trjconv -f p*xtc -s p.tpr -o p_lipids_whole.xtc -n index.ndx -pbc whole', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
        p.communicate('Lipids\nLipids')
        p = subprocess.Popen('gmx convert-tpr -s p.tpr -o p_lipids.tpr -n index.ndx', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
        p.communicate('Lipids')
        p = subprocess.Popen('gmx editconf -f p_lipids.tpr -o p_lipids.gro', stdin=subprocess.PIPE, shell=True, universal_newlines=True)
            
        cmd = 'python3 {}/calc_lindemann_ori.py {}.txt p_lipids_whole.xtc p_lipids.gro 1 20000 100'.format(script_dir, f'/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP/{sys}')

        file_run = open('run.sh' ,'w')
        file_run.write(cmd)
        file_run.close()

rm: cannot remove 'p_lipids*': No such file or directory
                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'Lipids'
100000000 steps (2e+06 ps) remaining from first run.
Selected 0: 'Lipids'



GROMACS reminds you: "On average, it takes twenty years for the world's largest super computer to shrink down to the size of your laptop." (Pete Beckman)

                     :-) GROMACS - gmx editconf, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tielema

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 8568 atoms
Volume: 2915.3 nm^3, corresponds to roughly 1311800 electrons
Velocities found


rm: cannot remove 'p_lipids*': No such file or directory
                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   


...


GROMACS reminds you: "Ease Myself Into the Body Bag" (P.J. Harvey)

rm: cannot remove 'p_lipids*': No such file or directory
                     :-) GROMACS - gmx trjconv, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu V

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'Lipids'
100000000 steps (2e+06 ps) remaining from first run.
Selected 0: 'Lipids'


                   :-) GROMACS - gmx convert-tpr, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
   

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 8568 atoms
Volume: 2918.17 nm^3, corresponds to roughly 1313100 electrons
Velocities found


                     :-) GROMACS - gmx editconf, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
    